In [1]:
import numpy as np
import pandas as pd

In [13]:
from google.colab import drive
from google.colab import files

# path from your drive to data
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Feature Engineering

The datasets from Kaggle were imported, and the specifc files that were needed were selected. All games from the 2009/2010 season to the 2022/2023 season were compiled into one dataframe. Various advanced statistics were added based on the given box score data and the games were sorted chronologically. 

In [7]:
path = '/content/drive/MyDrive/College/Senior_Year/SP/BEM_Ec_120/Project/data/'

WTourneyData = pd.read_csv(path + 'WNCAATourneyDetailedResults.csv')
WRegularSeasonData = pd.read_csv(path + 'WRegularSeasonDetailedResults.csv')
all_games_data = pd.concat([WRegularSeasonData, WTourneyData])

WTeams = pd.read_csv(path + 'WTeams.csv')

In [8]:
# fcns to calculate game-by-game advanced stats

def field_goal_percentage(fgm, fga):
  return fgm/fga * 100

def three_pt_field_goal_percentage(fgm3, fga3):
  return fgm3/fga3 * 100

def true_shooting_percentage(pts, fga, fta):
  return pts / (2 * (fga + 0.475 * fta))

def eFG_percentage(fgm, fgm3, fga):
  return (fgm + 0.5 * fgm3) / fga

def three_pt_attempt_rate(fga3, fga):
  return fga3 / fga

def ft_attempt_rate(fta, fga):
  return fta / fga

def oreb_percentage(oreb, opp_dreb):
  return oreb / (oreb + opp_dreb)

def dreb_percentage(dreb, opp_oreb):
  return dreb / (dreb + opp_oreb)

def total_reb_percentage(oreb, dreb, opp_oreb, opp_dreb):
  tot_reb = oreb + dreb
  opp_tot_reb = opp_oreb + opp_dreb
  return tot_reb / (tot_reb + opp_tot_reb)

def total_reb(oreb, dreb):
  return oreb + dreb

def possessions(fga, fta, oreb, to):
  return (fga + 0.475 * fta - oreb + to)

def off_rating(fga, fta, oreb, to, score, opp_fga, opp_fta, opp_oreb, opp_to):
  poss = possessions(fga, fta, oreb, to)
  opp_poss = possessions(opp_fga, opp_fta, opp_oreb, opp_to)
  return 100 * score / (poss + opp_poss)

def def_rating(fga, fta, oreb, to, opp_fga, opp_fta, opp_oreb, opp_to, opp_score):
  poss = possessions(fga, fta, oreb, to)
  opp_poss = possessions(opp_fga, opp_fta, opp_oreb, opp_to)
  return 100 * opp_score / (poss + opp_poss)

def net_rating(fga, fta, oreb, to, score, opp_fga, opp_fta, opp_oreb, opp_to, opp_score):
  off_rtg = off_rating(fga, fta, oreb, to, score, opp_fga, opp_fta, opp_oreb, opp_to)
  def_rtg = def_rating(fga, fta, oreb, to, opp_fga, opp_fta, opp_oreb, opp_to, opp_score)
  return off_rtg - def_rtg

def off_efficiency(fga, fta, oreb, to, pts):
  poss = possessions(fga, fta, oreb, to)
  return 100 * pts / poss

def def_efficiency(fga, fta, oreb, to, opp_pts):
  poss = possessions(fga, fta, oreb, to)
  return 100 * opp_pts / poss

def net_efficiency(pts, fga, fta):
  return 100 * pts / (2 * (fga + 0.44*fta))

def point_differential(pts, opp_pts):
  return pts - opp_pts

def pace(num_ot, fga, fta, oreb, to, opp_fga, opp_fta, opp_oreb, opp_to):
  poss = possessions(fga, fta, oreb, to)
  opp_poss = possessions(opp_fga, opp_fta, opp_oreb, opp_to)
  team_mins = 40*5 + 5*5*num_ot # 40 minutes of 5 players on the court plus any additional overtimes 5 minutes in length with 5 players 
  # note that 40 is used to calculate this statistic in the WNBA but 48 is used in the NBA because of game length
  return 40 * (poss + opp_poss) / (2 * team_mins/5)

def pts_per_100(pts, fga, fta, oreb, to):
  poss = possessions(fga, fta, oreb, to)
  return 100 * pts / poss

def assist_rate(ast, fga, fta, to):
  return 100 * ast / (ast + fga + 0.44*fta + to)

def turnover_rate(ast, fga, fta, to):
  return 100 * to / (ast + fga + 0.44*fta + to)

def assist_to_turnover_ratio(ast, to):
  return ast/to

In [9]:
# Compute ELO

K = 20.
HOME_ADVANTAGE = 100

sorted_data = all_games_data.sort_values(['Season', 'DayNum'])

team_ids = set(sorted_data.WTeamID).union(set(sorted_data.LTeamID))

elo_dict = dict(zip(list(team_ids), [1500] * len(team_ids)))

sorted_data['margin'] = sorted_data.WScore - sorted_data.LScore

def elo_pred(elo1, elo2):
    return(1. / (10. ** (-(elo1 - elo2) / 400.) + 1.))

def expected_margin(elo_diff):
    return((7.5 + 0.006 * elo_diff))

def elo_update(w_elo, l_elo, margin):
    elo_diff = w_elo - l_elo
    pred = elo_pred(w_elo, l_elo)
    mult = ((margin + 3.) ** 0.8) / expected_margin(elo_diff)
    update = K * mult * (1 - pred)
    return(pred, update)

preds = []
w_elo = []
l_elo = []

for row in sorted_data.itertuples():
  w = row.WTeamID
  l = row.LTeamID
  margin = row.margin
  wloc = row.WLoc
  w_ad, l_ad, = 0., 0.
  if wloc == "H":
      w_ad += HOME_ADVANTAGE
  elif wloc == "A":
      l_ad += HOME_ADVANTAGE
  pred, update = elo_update(elo_dict[w] + w_ad, elo_dict[l] + l_ad, margin)
  elo_dict[w] += update
  elo_dict[l] -= update
  preds.append(pred)
  w_elo.append(elo_dict[w])
  l_elo.append(elo_dict[l])
sorted_data['WELO'] = w_elo
sorted_data['LELO'] = l_elo

In [10]:
# calculate game-by-game advanced stats and place them in dataframe

wfga = sorted_data["WFGA"]
lfga = sorted_data["LFGA"]
wfgm = sorted_data["WFGM"]
lfgm = sorted_data["LFGM"]
wfga3 = sorted_data["WFGA3"]
lfga3 = sorted_data["LFGA3"]
wfgm3 = sorted_data["WFGM3"]
lfgm3 = sorted_data["LFGM3"]
wfta = sorted_data["WFTA"]
lfta = sorted_data["LFTA"]
wast = sorted_data["WAst"]
last = sorted_data["LAst"]
woreb = sorted_data["WOR"]
loreb = sorted_data["LOR"]
wdreb = sorted_data["WDR"]
ldreb = sorted_data["LDR"]
wto = sorted_data["WTO"]
lto = sorted_data["LTO"]
wpts = sorted_data["WScore"]
lpts = sorted_data["LScore"]
num_ot = sorted_data["NumOT"]


w_tsp = true_shooting_percentage(wpts, wfga, wfta)
l_tsp = true_shooting_percentage(lpts, lfga, lfta)

w_fgp = field_goal_percentage(wfgm, wfga)
l_fgp = field_goal_percentage(lfgm, lfga)

w_fgp3 = three_pt_field_goal_percentage(wfgm3, wfga3)
l_fgp3 = three_pt_field_goal_percentage(lfgm3, lfga3)

w_efgp = eFG_percentage(wfgm, wfgm3, wfga)
l_efgp = eFG_percentage(lfgm, lfgm3, lfga)

w_3pt_attempt_rate = three_pt_attempt_rate(wfga3, wfga)
l_3pt_attempt_rate = three_pt_attempt_rate(lfga3, lfga)

w_ft_attempt_rate = ft_attempt_rate(wfta, wfga)
l_ft_attempt_rate = ft_attempt_rate(lfta, lfga)

w_off_reb_percentage = oreb_percentage(woreb, ldreb)
l_off_reb_percentage = oreb_percentage(loreb, wdreb)

w_def_reb_percentage = dreb_percentage(wdreb, loreb)
l_def_reb_percentage = dreb_percentage(ldreb, woreb)

w_tot_reb_percentage = total_reb_percentage(woreb, wdreb, loreb, ldreb)
l_tot_reb_percentage = total_reb_percentage(loreb, ldreb, woreb, wdreb)

w_tot_reb = total_reb(woreb, wdreb)
l_tot_reb = total_reb(loreb, ldreb)

w_possessions = possessions(wfga, wfta, woreb, wto)
l_possessions = possessions(lfga, lfta, loreb, lto)

w_off_rating = off_rating(wfga, wfta, woreb, wto, wpts, lfga, lfta, loreb, lto)
l_off_rating = off_rating(lfga, lfta, loreb, lto, lpts, wfga, wfta, woreb, wto)

w_def_rating = def_rating(wfga, wfta, woreb, wto, lfga, lfta, loreb, lto, lpts)
l_def_rating = def_rating(lfga, lfta, loreb, lto, wfga, wfta, woreb, wto, wpts)

w_net_rating = net_rating(wfga, wfta, woreb, wto, wpts, lfga, lfta, loreb, lto, lpts)
l_net_rating = net_rating(lfga, lfta, loreb, lto, lpts, wfga, wfta, woreb, wto, wpts)

w_off_efficiency = off_efficiency(wfga, wfta, woreb, wto, wpts)
l_off_efficiency = off_efficiency(lfga, lfta, loreb, lto, lpts)

w_def_efficiency = def_efficiency(lfga, lfta, loreb, lto, lpts)
l_def_efficiency = def_efficiency(wfga, wfta, woreb, wto, wpts)

w_net_efficiency = net_efficiency(wpts, wfga, wfta)
l_net_efficiency = net_efficiency(lpts, lfga, lfta)

w_point_differential = point_differential(wpts, lpts)
l_point_differential = point_differential(lpts, wpts)

w_pace = pace(num_ot, wfga, wfta, woreb, wto, lfga, lfta, loreb, lto)
l_pace = pace(num_ot, lfga, lfta, loreb, lto, wfga, wfta, woreb, wto)

w_points_per_100 = pts_per_100(wpts, wfga, wfta, woreb, wto)
l_points_per_100 = pts_per_100(lpts, lfga, lfta, loreb, lto)

w_assist_rate = assist_rate(wast, wfga, wfta, wto)
l_assist_rate = assist_rate(last, lfga, lfta, lto)

w_turnover_rate = turnover_rate(wast, wfga, wfta, wto)
l_turnover_rate = turnover_rate(last, lfga, lfta, lto)

w_assist_to_turnover_ratio = assist_to_turnover_ratio(wast, wto)
l_assist_to_turnover_ratio = assist_to_turnover_ratio(last, lto)

# add stats to one dataframe (use sorted_data so it already has elo)

sorted_data["WTSP"] = w_tsp
sorted_data["LTSP"] = l_tsp
sorted_data["WFGp"] = w_fgp
sorted_data["LFGp"] = l_fgp
sorted_data["WFGp3"] = np.nan_to_num(w_fgp3, 0)
sorted_data["LFGp3"] = np.nan_to_num(l_fgp3, 0)
sorted_data["WEFGp"] = w_efgp 
sorted_data["LEFGp"] = l_efgp
sorted_data["W3PTAr"] = w_3pt_attempt_rate
sorted_data["L3PTAr"] = l_3pt_attempt_rate
sorted_data["WFTAr"] = w_ft_attempt_rate
sorted_data["LFTAr"] = l_ft_attempt_rate 
sorted_data["WORp"] = w_off_reb_percentage
sorted_data["LORp"] = l_off_reb_percentage 
sorted_data["WDRp"] = w_def_reb_percentage
sorted_data["LDRp"] = l_def_reb_percentage
sorted_data["WTRp"] = w_tot_reb_percentage
sorted_data["LTRp"] = l_tot_reb_percentage
sorted_data["WTR"] = w_tot_reb
sorted_data["LTR"] = l_tot_reb
sorted_data["WPoss"] = w_possessions
sorted_data["LPoss"] = l_possessions
sorted_data["WOffRtg"] = w_off_rating 
sorted_data["LOffRtg"] = l_off_rating
sorted_data["WDefRtg"] = w_def_rating
sorted_data["LDefRtg"] = l_def_rating
sorted_data["WNetRtg"] = w_net_rating 
sorted_data["LNetRtg"] = l_net_rating 
sorted_data["WOffEff"] = w_off_efficiency
sorted_data["LOffEff"] = l_off_efficiency
sorted_data["WDefEff"] = w_def_efficiency 
sorted_data["LDefEff"] = l_def_efficiency
sorted_data["WNetEff"] = w_net_efficiency
sorted_data["LNetEff"] = l_net_efficiency
sorted_data["WPointDiff"] = w_point_differential
sorted_data["LPointDiff"] = l_point_differential
sorted_data["WPace"] = w_pace
sorted_data["LPace"] = l_pace
sorted_data["WPtsPer100"] = w_points_per_100 
sorted_data["LPtsPer100"] = l_points_per_100 
sorted_data["WAr"] = w_assist_rate
sorted_data["LAr"] = l_assist_rate 
sorted_data["WTOr"] = w_turnover_rate 
sorted_data["LTOr"] = l_turnover_rate
sorted_data["WATRatio"] = w_assist_to_turnover_ratio
sorted_data["LATRatio"] = l_assist_to_turnover_ratio 

In [12]:
# Compute Pythagorean expectation 

def pythagorean_exp_1(pts, opp_pts):
  return (pts**16.5) / (pts**16.5 + opp_pts**16.5)

def pythagorean_exp_2(pts, opp_pts):
  return (pts**13.91) / (pts**13.91 + opp_pts**13.91)

START_SEASON = min(all_games_data['Season'])
END_SEASON = max(all_games_data['Season'])
SEASONS = [*range(START_SEASON, END_SEASON+1)]

TEAMS = WTeams['TeamID']
teams = list(TEAMS)
# the following variables are lists (seasons) of lists (stat for each team in order of TEAMS)
total_pts_for = []
total_pts_against = []

for season in SEASONS:
  season_pts_for = []
  season_pts_against = []
  for team in TEAMS:
    pts_for_losses = list(all_games_data.loc[(all_games_data['LTeamID']==team) & (all_games_data['Season']==season)]["LScore"])
    pts_for_wins = list(all_games_data.loc[(all_games_data['WTeamID']==team) & (all_games_data['Season']==season)]["WScore"])
    pts_against_losses = list(all_games_data.loc[(all_games_data['LTeamID']==team) & (all_games_data['Season']==season)]["WScore"])
    pts_against_wins = list(all_games_data.loc[(all_games_data['WTeamID']==team) & (all_games_data['Season']==season)]["LScore"])
    pts_for = sum(pts_for_wins) + sum(pts_for_losses)
    pts_against = sum(pts_against_wins) + sum(pts_against_losses)
    season_pts_for.append(pts_for)
    season_pts_against.append(pts_against)
  total_pts_for.append(season_pts_for)
  total_pts_against.append(season_pts_against)

# the pythagorean expectations are organized as a list (by season) of lists (where each is each team's stat for said season)
pythagorean_expectation_1 = []
pythagorean_expectation_2 = []
for s_ind in range(len(SEASONS)):
  season_stat1 = []
  season_stat2 = []
  for t_ind in range(len(TEAMS)):
    pts_for = total_pts_for[s_ind][t_ind]
    pts_against = total_pts_against[s_ind][t_ind]
    if pts_for != 0 or pts_against != 0:
      stat1 = pythagorean_exp_1(pts_for, pts_against)
      stat2 = pythagorean_exp_2(pts_for, pts_against)
    elif pts_for == 0 and pts_against == 0:
      stat1 = 0
      stat2 = 0
    season_stat1.append(stat1)
    season_stat2.append(stat2)
  pythagorean_expectation_1.append(season_stat1)
  pythagorean_expectation_2.append(season_stat2)


w_pythag_exp1 = []
l_pythag_exp1 = []
w_pythag_exp2 = []
l_pythag_exp2 = []

for row in sorted_data.itertuples():
  winner = row.WTeamID
  loser = row.LTeamID
  season = row.Season
  winner_index = teams.index(winner)
  loser_index = teams.index(loser)
  season_index = SEASONS.index(season)
  winner_stat1 = pythagorean_expectation_1[season_index - 1][winner_index]
  loser_stat1 = pythagorean_expectation_1[season_index - 1][loser_index]
  winner_stat2 = pythagorean_expectation_2[season_index - 1][winner_index]
  loser_stat2 = pythagorean_expectation_2[season_index - 1][loser_index]
  w_pythag_exp1.append(winner_stat1)
  l_pythag_exp1.append(loser_stat1)
  w_pythag_exp2.append(winner_stat2)
  l_pythag_exp2.append(loser_stat2)

sorted_data["WPythagoreanExp1"] = w_pythag_exp1
sorted_data["LPythagoreanExp1"] = l_pythag_exp1
sorted_data["WPythagoreanExp2"] = w_pythag_exp2
sorted_data["LPythagoreanExp2"] = l_pythag_exp2

In [14]:
# Save sorted_data for future use

sorted_data.to_csv("sorted_data.csv")
files.download("sorted_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Data Frame Construction

The model is meant to predict games off of past game data. To do this, it will predict off of the average stats of the two teams playing over the past n games. The number of games averaged over and stats selected will be optimally chosen as a pair. Thus, dataframes must be constructed with the same seasons, team matchups, and game outcomes but with the box score statistics averaged over the past n games (not including that game) so that the model can be trained on this and predict off of past averages accurately. 

In [ ]:
def get_avg_stats_last_n_games(team, season, daynum, n) :
  past_season_df = sorted_data[sorted_data['Season'] < season]
  past_season_df = past_season_df[(past_season_df['WTeamID'] == team) | (past_season_df['LTeamID'] == team)]
  curr_season_df = sorted_data[sorted_data['Season'] == season]
  curr_season_df = curr_season_df[(curr_season_df['WTeamID'] == team) | (curr_season_df['LTeamID'] == team)]
  curr_season_df = curr_season_df[curr_season_df["DayNum"] < daynum]
  prev_game_df = pd.concat([past_season_df.reset_index(drop=True), curr_season_df])
  prev_n_df = prev_game_df.tail(n)

  w_df = prev_n_df[prev_n_df["WTeamID"] == team]
  w_df = w_df[['Season', 'WScore',
                       'WFGM', 'WFGA', 'WFGp',
                       'WFGM3', 'WFGA3', 'WFGp3', 'WFTM',
                       'WFTA', 'WOR', 'WDR', 'WAst',
                       'WTO', 'WStl', 'WBlk', 'WPF',
                       'WTSP', 'WEFGp', 'W3PTAr',
                       'WFTAr', 'WORp', 'WDRp', 'WTR', 'WPoss',
                       'WOffRtg', 'WDefRtg', 'WNetRtg',
                       'WNetRtg', 'WOffEff', 'WDefEff',
                       'WNetEff', 'WPointDiff', 'WPace',
                       'WPtsPer100', 'WAr', 'WTOr',
                       'WATRatio', 'WELO', 'WPythagoreanExp1', 'WPythagoreanExp2']]
  l_df = prev_n_df[prev_n_df["LTeamID"] == team]
  l_df = l_df[['Season', 'LScore',
                       'LFGM', 'LFGA', 'LFGp',
                       'LFGM3', 'LFGA3', 'LFGp3', 'LFTM',
                       'LFTA', 'LOR', 'LDR', 'LAst',
                       'LTO', 'LStl', 'LBlk', 'LPF',
                       'LTSP', 'LEFGp', 'L3PTAr',
                       'LFTAr', 'LORp', 'LDRp', 'WTR', 'LPoss',
                       'LOffRtg', 'LDefRtg', 'LNetRtg',
                       'LNetRtg', 'LOffEff', 'LDefEff',
                       'LNetEff', 'LPointDiff', 'LPace',
                       'LPtsPer100', 'LAr', 'LTOr',
                       'LATRatio', 'LELO', 'LPythagoreanExp1', 'LPythagoreanExp2']]
  cols = ['Season', 'Score', 'FGM', 
        'FGA', 'FGp', 'FGM3', 'FGA3', 'FGp3', 'FTM', 
        'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
        'Blk', 'PF', 'TSP', 'EFGp', '3PTAr', 'FTAr', 
        'ORp', 'DRp', 'TRp', 'TR', 'Poss', 'OffRtg', 'DefRtg',
        'NetRtg', 'OffEff', 'DefEff', 'NetEff', 
        'PointDiff', 'Pace','PtsPer100', 'Ar', 'TOr', 
        'ATRatio', 'ELO', 'PythagoreanExp1', 'PythagoreanExp2']
  w_df.columns = cols
  l_df.columns = cols
  df = pd.concat([w_df.reset_index(drop=True), l_df])
  df.drop(columns = ['Season'], inplace=True)

  return df.mean()


def get_prev_game_info(season, daynum, team):
  past_season_df = sorted_data[sorted_data['Season'] < season]
  past_season_df = past_season_df[(past_season_df['WTeamID'] == team) | (past_season_df['LTeamID'] == team)]
  curr_season_df = sorted_data[sorted_data['Season'] == season]
  curr_season_df = curr_season_df[(curr_season_df['WTeamID'] == team) | (curr_season_df['LTeamID'] == team)]
  curr_season_df = curr_season_df[curr_season_df["DayNum"] < daynum]
  prev_game_df = pd.concat([past_season_df.reset_index(drop=True), curr_season_df])
  last_game = prev_game_df.tail(1)
  if last_game.shape[0] == 0:
    return pd.Series([0]*40, index =['Score', 'FGM', 
        'FGA', 'FGp', 'FGM3', 'FGA3', 'FGp3', 'FTM', 
        'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
        'Blk', 'PF', 'TSP', 'EFGp', '3PTAr', 'FTAr', 
        'ORp', 'DRp', 'TRp', 'TR', 'Poss', 'OffRtg', 'DefRtg',
        'NetRtg', 'OffEff', 'DefEff', 'NetEff', 
        'PointDiff', 'Pace','PtsPer100', 'Ar', 'TOr', 
        'ATRatio', 'ELO', 'PythagoreanExp1', 'PythagoreanExp2'])
  if last_game["WTeamID"] == team:
    return last_game[['WScore',
                       'WFGM', 'WFGA', 'WFGp',
                       'WFGM3', 'WFGA3', 'WFGp3', 'WFTM',
                       'WFTA', 'WOR', 'WDR', 'WAst',
                       'WTO', 'WStl', 'WBlk', 'WPF',
                       'WTSP', 'WEFGp', 'W3PTAr',
                       'WFTAr', 'WORp', 'WDRp', 'WTR', 'WPoss',
                       'WOffRtg', 'WDefRtg', 'WNetRtg',
                       'WNetRtg', 'WOffEff', 'WDefEff',
                       'WNetEff', 'WPointDiff', 'WPace',
                       'WPtsPer100', 'WAr', 'WTOr',
                       'WATRatio', 'WELO', 'WPythagoreanExp1', 'WPythagoreanExp2']]
  if last_game["LTeamID"] == team:
    return last_game[['LScore',
                       'LFGM', 'LFGA', 'LFGp',
                       'LFGM3', 'LFGA3', 'LFGp3', 'LFTM',
                       'LFTA', 'LOR', 'LDR', 'LAst',
                       'LTO', 'LStl', 'LBlk', 'LPF',
                       'LTSP', 'LEFGp', 'L3PTAr',
                       'LFTAr', 'LORp', 'LDRp', 'WTR', 'LPoss',
                       'LOffRtg', 'LDefRtg', 'LNetRtg',
                       'LNetRtg', 'LOffEff', 'LDefEff',
                       'LNetEff', 'LPointDiff', 'LPace',
                       'LPtsPer100', 'LAr', 'LTOr',
                       'LATRatio', 'LELO', 'LPythagoreanExp1', 'LPythagoreanExp2']]

In [ ]:
# Create dataframe of averages from last n games
# All data is stored in avg_matchups_n_data

n = {1,5,6,7,8,9,10,11,12,13,14,15,20,30}

for N in n:
  df_avg_matchups = sorted_data[["Season","DayNum","WTeamID","LTeamID"]]
  df_avg_matchups = df_avg_matchups.loc[(df_avg_matchups["Season"] != 2010)]

  wscore = []
  lscore = []
  wfgm = []
  lfgm = []
  wfga = []
  lfga = []
  wfgp = []
  lfgp = []
  wfgm3 = []
  lfgm3 = []
  wfga3 = []
  lfga3 = []
  wfgp3 = []
  lfgp3 = []
  wftm = []
  lftm = []
  wfta = []
  lfta = []
  wor = []
  lor = []
  wdr = []
  ldr = []
  wast = []
  last = []
  wto = []
  lto = []
  wstl = []
  lstl = []
  wblk = []
  lblk = []
  wpf = []
  lpf = []
  wtsp = []
  ltsp = []
  wefgp = []
  lefgp = []
  w3ptar = []
  l3ptar = []
  wftar = []
  lftar = []
  worp = []
  lorp = []
  wdrp = []
  ldrp = []
  wtrp = []
  ltrp = []
  wtr = []
  ltr = []
  wposs = []
  lposs = []
  woffrtg = []
  loffrtg = []
  wdefrtg = []
  ldefrtg = []
  wnetrtg = []
  lnetrtg = []
  woffeff = []
  loffeff = []
  wdefeff = []
  ldefeff = []
  wneteff = []
  lneteff = []
  wptdiff = []
  lptdiff = []
  wpace = []
  lpace = []
  wpp100 = []
  lpp100 = []
  war = []
  lar = []
  wtor = []
  ltor = []
  watratio = []
  latratio = []
  welo = []
  lelo = []
  wpyth1 = []
  lpyth1 = []
  wpyth2 = []
  lpyth2 = []

  for row in df_avg_matchups.itertuples():
    wteam_avgs = get_avg_stats_last_n_games(row.WTeamID, row.Season, row.DayNum, N)
    if pd.isna(wteam_avgs["Score"]):
      game_info = get_prev_game_info(row.Season, row.DayNum, row.WTeamID)
      wteam_avgs = game_info
      wteam_avgs.columns = ['Score', 'FGM', 
          'FGA', 'FGp', 'FGM3', 'FGA3', 'FGp3', 'FTM', 
          'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
          'Blk', 'PF', 'TSP', 'EFGp', '3PTAr', 'FTAr', 
          'ORp', 'DRp', 'TRp', 'TR', 'Poss', 'OffRtg', 'DefRtg',
          'NetRtg', 'OffEff', 'DefEff', 'NetEff', 
          'PointDiff', 'Pace','PtsPer100', 'Ar', 'TOr', 
          'ATRatio', 'ELO', 'PythagoreanExp1', 'PythagoreanExp2']
    lteam_avgs = get_avg_stats_last_n_games(row.LTeamID, row.Season, row.DayNum, N)
    if pd.isna(lteam_avgs["Score"]):
      game_info = get_prev_game_info(row.Season, row.DayNum, row.LTeamID)
      lteam_avgs = game_info
      lteam_avgs.columns = ['Score', 'FGM', 
          'FGA', 'FGp', 'FGM3', 'FGA3', 'FGp3', 'FTM', 
          'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl',
          'Blk', 'PF', 'TSP', 'EFGp', '3PTAr', 'FTAr', 
          'ORp', 'DRp', 'TRp', 'TR', 'Poss', 'OffRtg', 'DefRtg',
          'NetRtg', 'OffEff', 'DefEff', 'NetEff', 
          'PointDiff', 'Pace','PtsPer100', 'Ar', 'TOr', 
          'ATRatio', 'ELO', 'PythagoreanExp1', 'PythagoreanExp2']
    wscore.append(wteam_avgs["Score"])
    lscore.append(lteam_avgs["Score"])
    wfgm.append(wteam_avgs["FGM"])
    lfgm.append(lteam_avgs["FGM"])
    wfga.append(wteam_avgs["FGA"])
    lfga.append(lteam_avgs["FGA"])
    wfgp.append(wteam_avgs["FGp"])
    lfgp.append(lteam_avgs["FGp"])
    wfgm3.append(wteam_avgs["FGM3"])
    lfgm3.append(lteam_avgs["FGM3"])
    wfga3.append(wteam_avgs["FGA3"])
    lfga3.append(lteam_avgs["FGA3"])
    wfgp3.append(wteam_avgs["FGp3"])
    lfgp3.append(lteam_avgs["FGp3"])
    wftm.append(wteam_avgs["FTM"])
    lftm.append(lteam_avgs["FTM"])
    wfta.append(wteam_avgs["FTA"])
    lfta.append(lteam_avgs["FTA"])
    wor.append(wteam_avgs["OR"])
    lor.append(lteam_avgs["OR"])
    wdr.append(wteam_avgs["DR"])
    ldr.append(lteam_avgs["DR"])
    wast.append(wteam_avgs["Ast"])
    last.append(lteam_avgs["Ast"])
    wto.append(wteam_avgs["TO"])
    lto.append(lteam_avgs["TO"])
    wstl.append(wteam_avgs["Stl"])
    lstl.append(lteam_avgs["Stl"])
    wblk.append(wteam_avgs["Blk"])
    lblk.append(lteam_avgs["Blk"])
    wpf.append(wteam_avgs["PF"])
    lpf.append(lteam_avgs["PF"])
    wtsp.append(wteam_avgs["TSP"])
    ltsp.append(lteam_avgs["TSP"])
    wefgp.append(wteam_avgs["EFGp"])
    lefgp.append(lteam_avgs["EFGp"])
    w3ptar.append(wteam_avgs["3PTAr"])
    l3ptar.append(lteam_avgs["3PTAr"])
    wftar.append(wteam_avgs["FTAr"])
    lftar.append(lteam_avgs["FTAr"])
    worp.append(wteam_avgs["ORp"])
    lorp.append(lteam_avgs["ORp"])
    wdrp.append(wteam_avgs["DRp"])
    ldrp.append(lteam_avgs["DRp"])
    wtrp.append(wteam_avgs["TRp"])
    ltrp.append(lteam_avgs["TRp"])
    wtr.append(wteam_avgs["TR"])
    ltr.append(lteam_avgs["TR"])
    wposs.append(wteam_avgs["Poss"])
    lposs.append(lteam_avgs["Poss"])
    woffrtg.append(wteam_avgs["OffRtg"])
    loffrtg.append(lteam_avgs["OffRtg"])
    wdefrtg.append(wteam_avgs["DefRtg"])
    ldefrtg.append(lteam_avgs["DefRtg"])
    wnetrtg.append(wteam_avgs["NetRtg"])
    lnetrtg.append(lteam_avgs["NetRtg"])
    woffeff.append(wteam_avgs["OffEff"])
    loffeff.append(lteam_avgs["OffEff"])
    wdefeff.append(wteam_avgs["DefEff"])
    ldefeff.append(lteam_avgs["DefEff"])
    wneteff.append(wteam_avgs["NetEff"])
    lneteff.append(lteam_avgs['NetEff'])
    wptdiff.append(wteam_avgs["PointDiff"])
    lptdiff.append(lteam_avgs["PointDiff"])
    wpace.append(wteam_avgs["Pace"])
    lpace.append(lteam_avgs["Pace"])
    wpp100.append(wteam_avgs["PtsPer100"])
    lpp100.append(lteam_avgs["PtsPer100"])
    war.append(wteam_avgs["Ar"])
    lar.append(lteam_avgs["Ar"])
    wtor.append(wteam_avgs["TOr"])
    ltor.append(lteam_avgs["TOr"])
    watratio.append(wteam_avgs["ATRatio"])
    latratio.append(lteam_avgs["ATRatio"])
    welo.append(wteam_avgs["ELO"])
    lelo.append(lteam_avgs["ELO"])
    wpyth1.append(wteam_avgs["PythagoreanExp1"])
    lpyth1.append(lteam_avgs["PythagoreanExp1"])
    wpyth2.append(wteam_avgs["PythagoreanExp2"])
    lpyth2.append(lteam_avgs["PythagoreanExp2"])

  df_avg_matchups["WScore"] = wscore 
  df_avg_matchups["LScore"] = lscore
  df_avg_matchups["WFGM"] = wfgm
  df_avg_matchups["LFGM"] = lfgm
  df_avg_matchups["WFGA"] = wfga
  df_avg_matchups["LFGA"] = lfga
  df_avg_matchups["WFGp"] = wfgp
  df_avg_matchups["LFGp"] = lfgp
  df_avg_matchups["WFGM3"] = wfgm3
  df_avg_matchups["LFGM3"] = lfgm3
  df_avg_matchups["WFGA3"] = wfga3 
  df_avg_matchups["LFGA3"] = lfga3
  df_avg_matchups["WFGp3"] = wfgp3 
  df_avg_matchups["LFGp3"] = lfgp3
  df_avg_matchups["WFTM"] = wftm
  df_avg_matchups["LFTM"] = lftm
  df_avg_matchups["WFTA"] = wfta
  df_avg_matchups["LFTA"] = lfta 
  df_avg_matchups["WOR"] = wor
  df_avg_matchups["LOR"] = lor 
  df_avg_matchups["WDR"] = wdr
  df_avg_matchups["LDR"] = ldr 
  df_avg_matchups["WAst"] = wast 
  df_avg_matchups["LAst"] = last
  df_avg_matchups["WTO"] = wto
  df_avg_matchups["LTO"] = lto 
  df_avg_matchups["WStl"] = wstl 
  df_avg_matchups["LStl"] = lstl
  df_avg_matchups["WBlk"] = wblk
  df_avg_matchups["LBlk"] = lblk
  df_avg_matchups["WPF"] = wpf
  df_avg_matchups["LPF"] = lpf
  df_avg_matchups["WTSP"] = wtsp
  df_avg_matchups["LTSP"] = ltsp 
  df_avg_matchups["WEFGp"] = wefgp
  df_avg_matchups["LEFGp"] = lefgp 
  df_avg_matchups["W3PTAr"] = w3ptar
  df_avg_matchups["L3PTAr"] = l3ptar 
  df_avg_matchups["WFTAr"] = wftar
  df_avg_matchups["LFTAr"] = lftar
  df_avg_matchups["WORp"] = worp 
  df_avg_matchups["LORp"] = lorp 
  df_avg_matchups["WDRp"] = wdrp
  df_avg_matchups["LDRp"] = ldrp
  df_avg_matchups["WTRp"] = wtrp 
  df_avg_matchups["LTRp"] = ltrp
  df_avg_matchups["WTR"] = wtr
  df_avg_matchups["LTR"] = ltr
  df_avg_matchups["WPoss"] = wposs
  df_avg_matchups["LPoss"] = lposs
  df_avg_matchups["WOffRtg"] = woffrtg
  df_avg_matchups["LOffRtg"] = loffrtg 
  df_avg_matchups["WDefRtg"] = wdefrtg
  df_avg_matchups["LDefRtg"] = ldefrtg
  df_avg_matchups["WNetRtg"] = wnetrtg 
  df_avg_matchups["LNetRtg"] = lnetrtg
  df_avg_matchups["WOffEff"] = woffeff
  df_avg_matchups["LOffEff"] = loffeff
  df_avg_matchups["WDefEff"] = wdefeff 
  df_avg_matchups["LDefEff"] = ldefeff
  df_avg_matchups["WNetEff"] = wneteff
  df_avg_matchups["LNetEff"] = lneteff 
  df_avg_matchups["WPointDiff"] = wptdiff 
  df_avg_matchups["LPointDiff"] = lptdiff 
  df_avg_matchups["WPace"] = wpace 
  df_avg_matchups["LPace"] = lpace 
  df_avg_matchups["WPtsPer100"] = wpp100 
  df_avg_matchups["LPtsPer100"] = lpp100 
  df_avg_matchups["WAr"] = war
  df_avg_matchups["LAr"] = lar 
  df_avg_matchups["WTOr"] = wtor
  df_avg_matchups["LTOr"] = ltor
  df_avg_matchups["WATRatio"] = watratio 
  df_avg_matchups["LATRatio"] = latratio
  df_avg_matchups["WELO"] = welo 
  df_avg_matchups["LELO"] = lelo 
  df_avg_matchups["WPythagoreanExp1"] = wpyth1 
  df_avg_matchups["LPythagoreanExp1"] = lpyth1 
  df_avg_matchups["WPythagoreanExp2"] = wpyth2
  df_avg_matchups["LPythagoreanExp2"] = lpyth2

  df_avg_matchups.to_csv("avg_matchups_n={}.csv".format(N))
  files.download("avg_matchups_n={}.csv".format(N))